# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
spark.version

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.68:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1574419081464)
SparkSession available as 'spark'


res0: String = 2.4.4


In [2]:
val housing = spark.read.option("header","true").option("inferSchema","true").format("csv").load("data/housing.csv")

housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]


---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [3]:
housing.count()

res1: Long = 20640


In [4]:
housing.getClass

res2: Class[_ <: org.apache.spark.sql.DataFrame] = class org.apache.spark.sql.Dataset


## 2.2. Look at the data
Print the first five records of the dataset.

In [5]:
housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [3]:
housing.groupBy("population").count().filter("population > 10000").count()

res1: Long = 23


## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [7]:
housing.describe("housing_median_age").show(false)
housing.describe("total_rooms").show(false)
housing.describe("median_house_value").show(false)
housing.describe("population").show(false)

+-------+------------------+
|summary|housing_median_age|
+-------+------------------+
|count  |20640             |
|mean   |28.639486434108527|
|stddev |12.58555761211163 |
|min    |1.0               |
|max    |52.0              |
+-------+------------------+

+-------+------------------+
|summary|total_rooms       |
+-------+------------------+
|count  |20640             |
|mean   |2635.7630813953488|
|stddev |2181.6152515827944|
|min    |2.0               |
|max    |39320.0           |
+-------+------------------+

+-------+------------------+
|summary|median_house_value|
+-------+------------------+
|count  |20640             |
|mean   |206855.81690891474|
|stddev |115395.61587441359|
|min    |14999.0           |
|max    |500001.0          |
+-------+------------------+

+-------+------------------+
|summary|population        |
+-------+------------------+
|count  |20640             |
|mean   |1425.4767441860465|
|stddev |1132.46212176534  |
|min    |3.0               |
|max    |35

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [7]:
import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age")).show()
housing.agg(min("total_rooms")).show()
housing.agg(avg("median_house_value")).show()

+-----------------------+
|max(housing_median_age)|
+-----------------------+
|                   52.0|
+-----------------------+

+----------------+
|min(total_rooms)|
+----------------+
|             2.0|
+----------------+

+-----------------------+
|avg(median_house_value)|
+-----------------------+
|     206855.81690891474|
+-----------------------+



import org.apache.spark.sql.functions._


## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [4]:
housing.groupBy("ocean_proximity").count().sort(desc("count")).show(false)

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|<1H OCEAN      |9136 |
|INLAND         |6551 |
|NEAR OCEAN     |2658 |
|NEAR BAY       |2290 |
|ISLAND         |5    |
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [9]:
housing.groupBy("ocean_proximity").agg(avg("median_house_value")).withColumnRenamed("avg(median_house_value)","avg_value").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [10]:
housing.createOrReplaceTempView("df")

spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [5]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler()
                .setInputCols(Array("housing_median_age", "total_rooms", "median_house_value","population"))
                .setOutputCol("output")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|              output|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_597b0c9edb5a
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]


In [13]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "output").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [6]:
val housingCol1 = housing.withColumn("rooms_per_household", $"total_rooms" / $"households") 
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", $"total_bedrooms" / $"total_rooms") 
val housingExtra = housingCol2.withColumn("population_per_household", $"population" / $"households") 

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [7]:
val renamedHousing = housingExtra.withColumnRenamed("median_house_value","label")

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [8]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [9]:
for (c <- colNum) {
    val missing_values = renamedHousing.filter(renamedHousing(c).isNull || 
            renamedHousing(c) === "" || renamedHousing(c) === " " || renamedHousing(c).isNaN).count() 
    printf("Missing values for " + c + ": " + missing_values + "\n") 
}

Missing values for longitude: 0
Missing values for latitude: 0
Missing values for housing_median_age: 0
Missing values for total_rooms: 0
Missing values for total_bedrooms: 207
Missing values for population: 0
Missing values for households: 0
Missing values for median_income: 0
Missing values for rooms_per_household: 0
Missing values for bedrooms_per_room: 207
Missing values for population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [10]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median")
                    .setInputCols(Array("total_bedrooms", "bedrooms_per_room"))
                    .setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))

// We want only the numerical columns - exclude the categorical ocean_proximity
val imputedHousing = imputer
    .fit(renamedHousing.select(colNum.head, colNum.tail: _*))
    .transform(renamedHousing.select(colNum.head, colNum.tail: _*))

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_6c99e0374ffb
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]


In [11]:
// Sanity check
for (c <- imputedHousing.columns) {
    printf(c + " Missing values: ")
    val missingCount = imputedHousing.filter(imputedHousing(c).isNull || 
            imputedHousing(c) === "" || imputedHousing(c).isNaN).count() 
    println(missingCount)
}

longitude Missing values: 0
latitude Missing values: 0
housing_median_age Missing values: 0
total_rooms Missing values: 0
total_bedrooms Missing values: 0
population Missing values: 0
households Missing values: 0
median_income Missing values: 0
rooms_per_household Missing values: 0
bedrooms_per_room Missing values: 0
population_per_household Missing values: 0


### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [15]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredHousing = va.transform(imputedHousing)
featuredHousing.select("features").show(5, false)

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.select("scaled_features").show(5, false)
// scaledHousing.show(5)

+---------------------------------------------------------------------------------------------------------------+
|features                                                                                                       |
+---------------------------------------------------------------------------------------------------------------+
|[-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,6.984126984126984,0.14659090909090908,2.5555555555555554]   |
|[-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,6.238137082601054,0.15579659106916466,2.109841827768014]|
|[-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,8.288135593220339,0.12951601908657123,2.8022598870056497]  |
|[-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,5.8173515981735155,0.18445839874411302,2.547945205479452]  |
|[-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,6.281853281853282,0.1720958819913952,2.1814671814671813]   |
+---------------------------------------------------------------------------------------

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_85753453109e
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_39a4c3e60022
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [16]:
renamedHousing.groupBy("ocean_proximity").count().sort(desc("count")).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [18]:
import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_Index")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.select("ocean_proximity_Index").show(5)

+---------------------+
|ocean_proximity_Index|
+---------------------+
|                  3.0|
|                  3.0|
|                  3.0|
|                  3.0|
|                  3.0|
+---------------------+
only showing top 5 rows



import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_6951fa3ff05b
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]


Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [19]:
indexer.fit(renamedHousing).labels

res15: Array[String] = Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)


### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [20]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("ocean_proximity_Index"))
    .setOutputCols(Array("ocean_proximity_one_hot"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(3)
ohHousing.select("ocean_proximity", "ocean_proximity_Index", "ocean_proximity_one_hot").show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_Index|ocean_proximity_one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                  3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     

import org.apache.spark.ml.feature.OneHotEncoderEstimator
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_d0954c99d868
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [22]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

// Pipeline for numerical values
val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
// Pipeline for categorical values
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(1)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|     scaled_features|ocean_proximity_Index|ocean_proximity_one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_74bb971cbc3d
catPipeline: org.apache.spark.ml.Pipeline = pipeline_de448fc875b0
pipeline: org.apache.spark.ml.Pipeline = pipeline_be90abcfcaea
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 15 more fields]


Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [32]:
newHousing.show(1)
val finalHousing = newHousing.drop("features")
finalHousing.show(1)

val va2 = new VectorAssembler()
    .setInputCols(Array("scaled_features", "ocean_proximity_one_hot"))
    .setOutputCol("features")

val dataset = va2.transform(finalHousing).select("features", "label")

dataset.show(1, false)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|     scaled_features|ocean_proximity_Index|ocean_proximity_one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090

finalHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 14 more fields]
va2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_e93b2aec141e
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: double]


---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [33]:
// TODO: Replace <FILL IN> with appropriate code

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet.show(5)
testSet.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-62.065401082150...| 94600.0|
|[-62.040445150874...| 85800.0|
|[-62.025471592109...| 79000.0|
|[-62.020480405854...|111400.0|
|[-62.015489219599...| 76100.0|
+--------------------+--------+
only showing top 5 rows

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-62.040445150874...|103600.0|
|[-61.985542102068...| 90100.0|
|[-61.985542102068...| 69000.0|
|[-61.985542102068...| 74600.0|
|[-61.980550915813...| 72200.0|
+--------------------+--------+
only showing top 5 rows



trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]


## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [35]:
import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr =  new LinearRegression()
      .setFeaturesCol("features")
      .setLabelCol("label")
      .setSolver("normal")
      .setRegParam(0.1)
      .setMaxIter(10)

val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-54046.43761479382,-54820.068788226425,13859.506105265367,4985.994972407238,4167.480664861903,-46822.53953798382,43159.45018520325,77114.35864047013,6544.2907454068845,14800.2630360449,914.5993855571517,-142332.35934356658,-178351.95133572767,-138987.78732683064,-145691.09403224153], Intercept: -2217059.1288151112
RMSE: 68221.90026448367


import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_9a52db0a9d63
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_9a52db0a9d63
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@b5df1e0


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [36]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction")

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 143024.0907437941|103600.0|[-62.040445150874...|
|191732.75871507684| 90100.0|[-61.985542102068...|
|170590.11375837168| 69000.0|[-61.985542102068...|
| 95902.06188121205| 74600.0|[-61.985542102068...|
|154761.69518025732| 72200.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66579.48866610067


import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_4ab8d1ff171e
rmse: Double = 66579.48866610067


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [37]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|172237.77863247864|103600.0|[-62.040445150874...|
| 229665.7083936324| 90100.0|[-61.985542102068...|
| 148492.6530612245| 69000.0|[-61.985542102068...|
|144325.54212932722| 74600.0|[-61.985542102068...|
|144325.54212932722| 72200.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67199.8013966304


import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_e1d91a7697f4
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel (uid=dtr_e1d91a7697f4) of depth 5 with 63 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_9b9918dbef60
rmse: Double = 67199.8013966304


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [39]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor()
                .setMaxDepth(10)
                .setNumTrees(25)
                .setLabelCol("label")
                .setFeaturesCol("features")
                .setFeatureSubsetStrategy("auto")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 119142.7535839752|103600.0|[-62.040445150874...|
| 167046.9538934113| 90100.0|[-61.985542102068...|
|104550.87507486211| 69000.0|[-61.985542102068...|
|  89673.7806055189| 74600.0|[-61.985542102068...|
| 86884.52451735463| 72200.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 53021.07682416412


import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_fc9e701ebfd4
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_fc9e701ebfd4) with 25 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_bbfdf7908d5b
rmse: Double = 53021.07682416412


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [40]:
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setLabelCol("label").setFeaturesCol("features").setMaxIter(10)

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator()
    .setMetricName("rmse")
    .setPredictionCol("prediction")
    .setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|129629.90520065554|103600.0|[-62.040445150874...|
| 158737.3570590097| 90100.0|[-61.985542102068...|
| 89222.40511241434| 69000.0|[-61.985542102068...|
|102692.65695644479| 74600.0|[-61.985542102068...|
|100080.94880496239| 72200.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 58340.26583695782


import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_63763a442462
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel (uid=gbtr_63763a442462) with 10 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_daeba8317b8b
rmse: Double = 58340.26583695782


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [41]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
                    .addGrid(rfModel.maxDepth, Array(1, 5, 10))
                    .addGrid(rfModel.numTrees, Array(5, 10, 15)).build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")

val pipeline = new Pipeline().setStages(Array(rfModel)) 
val cv = new CrossValidator().setEstimator(pipeline).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(3)
val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|  198571.255973292|103600.0|[-62.040445150874...|
|278411.58982235217| 90100.0|[-61.985542102068...|
|174251.45845810353| 69000.0|[-61.985542102068...|
|149456.30100919816| 74600.0|[-61.985542102068...|
| 144807.5408622577| 72200.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 159733.3094174567


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_fc9e701ebfd4-maxDepth: 1,
	rfr_fc9e701ebfd4-numTrees: 5
}, {
	rfr_fc9e701ebfd4-maxDepth: 1,
	rfr_fc9e701ebfd4-numTrees: 10
}, {
	rfr_fc9e701ebfd4-maxDepth: 1,
	rfr_fc9e701ebfd4-numTrees: 15
}, {
	rfr_fc9e701ebfd4-maxDepth: 5,
	rfr_fc9e701ebfd4-numTrees: 5
}, {
	rfr_fc9e701ebfd4-maxDepth: 5,
	rfr_fc9e701ebfd4-numTrees: 10
}, {
	rfr_fc9e701ebfd4-maxDepth: 5,
	rfr_fc9e701ebfd4-numTrees: 15
}, {
	rfr_fc9e701ebfd4-maxDepth: 10,
	rfr_fc9e701ebfd4-numTrees: 5
}, {
	rfr_fc9e701ebfd4-maxDepth: 10,
	rfr_fc9e701ebfd4-numTrees: 10
}, {
	rfr_fc9e701...

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [43]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "input1")
    final val inputCol2 = new Param[String](this, "inputCol2", "input2")
    final val outputCol = new Param[String](this, "outputCol", "output")
    
    override def copy(extra: ParamMap): MyParams = {
        defaultCopy(extra)
    }
    
    protected def validateAndTransformSchema(schema: StructType): StructType = {
        val idx1 = schema.fieldIndex($(inputCol1))
        val field1 = schema.fields(idx1)

        val idx2 = schema.fieldIndex($(inputCol2))
        val field2 = schema.fields(idx2)

        if (field1.dataType != DoubleType) {
          throw new Exception(s"Input type ${field1.dataType} did not match input type StringType")
        }
        if (field2.dataType != DoubleType) {
          throw new Exception(s"Input type ${field2.dataType} did not match input type StringType")
        }
        // Add the return field
        schema.add(StructField($(outputCol), DoubleType, false))  

    }
}


import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
defined trait MyParams


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [44]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}


class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("transformSchema"))
    
    def setInputCol1(value: String)= set(inputCol1, value)
    
    def setInputCol2(value: String)= set(inputCol2, value)
    
    def setOutputCol(value: String)= set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    } 

    override def transformSchema(schema: StructType): StructType = {
        validateAndTransformSchema(schema)
    }
      
    override def transform(dataset: Dataset[_]): DataFrame = {
        return dataset.select((col("total_rooms") / col("households")).alias("rooms_per_household"))
    }
}

import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}
defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [46]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

+-------------------+
|rooms_per_household|
+-------------------+
|  6.984126984126984|
|  6.238137082601054|
|  8.288135593220339|
| 5.8173515981735155|
|  6.281853281853282|
+-------------------+
only showing top 5 rows



myTransformer: MyTransformer = transformSchema_c69140fcc517
myDataset: Unit = ()


---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [39]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class Instance
defined object Helper


In [ ]:
class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

In [ ]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

In [ ]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

### Load the dataset

In [48]:
val dataset = spark.read.option("header","true").option("inferSchema","true").format("csv").load("data/ccdefault.csv")

dataset.printSchema()

val data = dataset.count
println(s"The data set has $data data points.")

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

dataset: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]
data: Long = 30000


### Delete ID column from the dataset

In [50]:
val dfDrop = dataset.drop("ID")

dfDrop: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]


### Summary of the attributes in the dataset

In [53]:
dfDrop.describe("LIMIT_BAL","AGE").show()

dataset.describe("PAY_0", "PAY_2","PAY_3","PAY_4","PAY_5","PAY_6").show()

// dataset.describe("BILL_AMT1", "BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6").show()

// dataset.describe("PAY_AMT1", "PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6", "DEFAULT").show()


+-------+------------------+-----------------+
|summary|         LIMIT_BAL|              AGE|
+-------+------------------+-----------------+
|  count|             30000|            30000|
|   mean|167484.32266666667|          35.4855|
| stddev|129747.66156720246|9.217904068090155|
|    min|             10000|               21|
|    max|           1000000|               79|
+-------+------------------+-----------------+

+-------+------------------+--------------------+------------------+--------------------+------------------+-----------------+
|summary|             PAY_0|               PAY_2|             PAY_3|               PAY_4|             PAY_5|            PAY_6|
+-------+------------------+--------------------+------------------+--------------------+------------------+-----------------+
|  count|             30000|               30000|             30000|               30000|             30000|            30000|
|   mean|           -0.0167|-0.13376666666666667|           -0.1662|

### Describe categorical features

In [55]:
val catFeaturesToDescribe = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2",
                               "PAY_3", "PAY_4", "PAY_5", "PAY_6")

for (feat <- catFeaturesToDescribe) {
    dfDrop.groupBy(feat)
        .count()
        .sort(desc("count"))
        .show()
}

+---+-----+
|SEX|count|
+---+-----+
|  2|18112|
|  1|11888|
+---+-----+

+---------+-----+
|EDUCATION|count|
+---------+-----+
|        2|14030|
|        1|10585|
|        3| 4917|
|        5|  280|
|        4|  123|
|        6|   51|
|        0|   14|
+---------+-----+

+--------+-----+
|MARRIAGE|count|
+--------+-----+
|       2|15964|
|       1|13659|
|       3|  323|
|       0|   54|
+--------+-----+

+-----+-----+
|PAY_0|count|
+-----+-----+
|    0|14737|
|   -1| 5686|
|    1| 3688|
|   -2| 2759|
|    2| 2667|
|    3|  322|
|    4|   76|
|    5|   26|
|    8|   19|
|    6|   11|
|    7|    9|
+-----+-----+

+-----+-----+
|PAY_2|count|
+-----+-----+
|    0|15730|
|   -1| 6050|
|    2| 3927|
|   -2| 3782|
|    3|  326|
|    4|   99|
|    1|   28|
|    5|   25|
|    7|   20|
|    6|   12|
|    8|    1|
+-----+-----+

+-----+-----+
|PAY_3|count|
+-----+-----+
|    0|15764|
|   -1| 5938|
|   -2| 4085|
|    2| 3819|
|    3|  240|
|    4|   76|
|    7|   27|
|    6|   23|
|    5|   21|
|

catFeaturesToDescribe: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)


### Analysis for the categorical features

From the tables that are presented above we can derive the following results:

1. We have more instances from females (18112) than males (11888)

2. The majority of the participants are graduates either from university (14030) or from school (10585), whereas only 4917 
   participants have graduated from high school. The minority has graduated from other institutes.

3. Almost half of the participants are single (15964), whereas 13659 are married and the rest 377 have undefined marital status.

4. PAY_0 is the attribute that describes the payment status on September, where as we can see from the measurement scale of the repayment status most of the participants (14737) paid without delay on September (measurement scale = 0), whereas the minority paid after 3 or more months. In the same way we can observe what were the results for the rest months (columns X7 to X11).

5. Although in all of those 7 months the majority of the participants paid without delay (measurement scale = 0), there is no other correlation in the rest of the scales. This means that one participant can have a different behavior between two or more different months. The only conclusion that we can derive, as it has been mentioned above, is that in all these 7 months the mojority paid without delay.

### Display the average limit balance for the different sexes, statuses and education levels

In [58]:
val featuresToDescribe = Array("SEX", "EDUCATION", "MARRIAGE")

for (feat <- featuresToDescribe) {
    dataset.groupBy(feat)
        .agg(avg("LIMIT_BAL"))
        .show()
}

+---+------------------+
|SEX|    avg(LIMIT_BAL)|
+---+------------------+
|  1| 163519.8250336474|
|  2|170086.46201413427|
+---+------------------+

+---------+------------------+
|EDUCATION|    avg(LIMIT_BAL)|
+---------+------------------+
|        1|212956.06991025034|
|        6|148235.29411764705|
|        3|126550.27049013626|
|        5| 168164.2857142857|
|        4|220894.30894308942|
|        2| 147062.4376336422|
|        0|217142.85714285713|
+---------+------------------+

+--------+------------------+
|MARRIAGE|    avg(LIMIT_BAL)|
+--------+------------------+
|       1|182200.89318398127|
|       3| 98080.49535603715|
|       2|156413.66073665748|
|       0|132962.96296296295|
+--------+------------------+



featuresToDescribe: Array[String] = Array(SEX, EDUCATION, MARRIAGE)


### Check for correlations 

In [62]:
import org.apache.spark.ml.linalg.{Vector, Vectors, Matrix, Matrices}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

def find_correlations(features: Array[String], df: DataFrame) = {
    val va = new VectorAssembler()
        .setInputCols(features)
        .setOutputCol("COMBINED_FEATURES")

    val attributes = va.transform(df)

    val Row(coeff: Matrix) = Correlation.corr(attributes, "COMBINED_FEATURES").head
    val matrixRows = coeff.rowIter.toSeq.map(_.toArray)
    val tempDf = spark.sparkContext.parallelize(matrixRows).toDF("Row")

    val numOfCols = matrixRows.head.length
    val dfCorrelation = (0 until numOfCols).foldLeft(tempDf)((tempDf, num) => 
        tempDf.withColumn("Col" + num, $"Row".getItem(num)))
      .drop("Row")

    println(s"The standard correlation coefficients:\n")
    dfCorrelation.show(false)
}

import org.apache.spark.ml.linalg.{Vector, Vectors, Matrix, Matrices}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
find_correlations: (features: Array[String], df: org.apache.spark.sql.DataFrame)Unit


In [63]:
// Find correlations in the bill_amt* features

val featuresBillAMT = Array("BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6")
val dfCorBillAMT = find_correlations(featuresBillAMT, dfDrop)

// Find correlations in the pay_amt* features
val featuresPayAMT = Array("PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6")
val dfCorPayAMT = find_correlations(featuresPayAMT, dfDrop)

// Find correlations in the pay_* features
val featuresPay = Array("PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")
val dfCorPay = find_correlations(featuresPay, dfDrop)

The standard correlation coefficients:

+------------------+------------------+------------------+------------------+------------------+------------------+
|Col0              |Col1              |Col2              |Col3              |Col4              |Col5              |
+------------------+------------------+------------------+------------------+------------------+------------------+
|1.0               |0.9514836727518164|0.8922785291271811|0.8602721890293089|0.8297786058329933|0.8026501885528523|
|0.9514836727518164|1.0               |0.9283262592714868|0.8924822912577247|0.8597783072714432|0.8315935591018226|
|0.8922785291271811|0.9283262592714868|1.0               |0.9239694565909823|0.8839096973620095|0.8533200905940505|
|0.8602721890293089|0.8924822912577247|0.9239694565909823|1.0               |0.9401344040880051|0.9009409547978421|
|0.8297786058329933|0.8597783072714432|0.8839096973620095|0.9401344040880051|1.0               |0.9461968070521906|
|0.8026501885528523|0.8315935591

featuresBillAMT: Array[String] = Array(BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6)
dfCorBillAMT: Unit = ()
featuresPayAMT: Array[String] = Array(PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
dfCorPayAMT: Unit = ()
featuresPay: Array[String] = Array(PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
dfCorPay: Unit = ()


In [66]:
dfDrop.show(1)
val dfTarget = dfDrop.withColumnRenamed("DEFAULT", "label")
dfTarget.show(1)
val dfDropped = dfTarget.drop("DEFAULT")
dfDropped.show(1)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
only showing to

dfTarget: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
dfDropped: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]


### Separate columns

In [67]:
// Label columns
val colLabel = "label"

// Feature columns
val colFeat = dfDropped.columns.filter(_ != colLabel)

// Categorical columns
val colCat = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3",
                   "PAY_4", "PAY_5", "PAY_6")

// Numerical columns
val colNum = Array("LIMIT_BAL", "AGE", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                   "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
                   "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6")

colLabel: String = label
colFeat: Array[String] = Array(LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
colCat: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
colNum: Array[String] = Array(LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)


### Make categorical features ready for one-hot encoding

In [68]:
// Make sure that the categorical variables don't contain any negative values
val df01 = dfDropped.na.replace("PAY_0", Map(-1 -> 10))
val df02 = df01.na.replace("PAY_0", Map(-2 -> 11))
val df21 = df02.na.replace("PAY_2", Map(-1 -> 10))
val df22 = df21.na.replace("PAY_2", Map(-2 -> 11))
val df31 = df22.na.replace("PAY_3", Map(-1 -> 10))
val df32 = df31.na.replace("PAY_3", Map(-2 -> 11))
val df41 = df32.na.replace("PAY_4", Map(-1 -> 10))
val df42 = df41.na.replace("PAY_4", Map(-2 -> 11))
val df51 = df42.na.replace("PAY_5", Map(-1 -> 10))
val df52 = df51.na.replace("PAY_5", Map(-2 -> 11))
val df61 = df52.na.replace("PAY_6", Map(-1 -> 10))
val dfUpdatedCatCols = df61.na.replace("PAY_6", Map(-2 -> 11))

df01: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df02: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df21: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df22: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df31: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df32: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df41: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df42: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df51: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df52: org.apache.spark....

### Check for missing values

In [69]:
for (c <- colFeat) {
    val missingCount = dfUpdatedCatCols.filter(dfDrop(c).isNull || 
            dfUpdatedCatCols(c) === "" || dfUpdatedCatCols(c).isNaN).count() 
    printf("Missing values for " + c + " : " + missingCount + "\n")
}

Missing values for LIMIT_BAL : 0
Missing values for SEX : 0
Missing values for EDUCATION : 0
Missing values for MARRIAGE : 0
Missing values for AGE : 0
Missing values for PAY_0 : 0
Missing values for PAY_2 : 0
Missing values for PAY_3 : 0
Missing values for PAY_4 : 0
Missing values for PAY_5 : 0
Missing values for PAY_6 : 0
Missing values for BILL_AMT1 : 0
Missing values for BILL_AMT2 : 0
Missing values for BILL_AMT3 : 0
Missing values for BILL_AMT4 : 0
Missing values for BILL_AMT5 : 0
Missing values for BILL_AMT6 : 0
Missing values for PAY_AMT1 : 0
Missing values for PAY_AMT2 : 0
Missing values for PAY_AMT3 : 0
Missing values for PAY_AMT4 : 0
Missing values for PAY_AMT5 : 0
Missing values for PAY_AMT6 : 0


### Scale the numerical variables

In [81]:
val va = new VectorAssembler().setInputCols(colNum).setOutputCol("FEATURES_TO_SCALE")

val dfToScale = va.transform(dfUpdatedCatCols)

val scaler = new StandardScaler().setInputCol("FEATURES_TO_SCALE").setOutputCol("SCALED_FEATURES")

val dfScaled = scaler.fit(dfToScale).transform(dfToScale)

dfScaled.select("SCALED_FEATURES").show(1, false)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|SCALED_FEATURES                                                                                                                             |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|(14,[0,1,2,3,4,9],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683])|
+--------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_9497e2b6ff3b
dfToScale: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 23 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_9fe67d9c358b
dfScaled: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 24 more fields]


### One-hot encode the categorical features

In [84]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator()
  .setInputCols(colCat)
  .setOutputCols(Array("SEX_one_hot", "EDUCATION_one_hot", "MARRIAGE_one_hot", "PAY_0_one_hot", "PAY_2_one_hot", "PAY_3_one_hot",
                   "PAY_4_one_hot", "PAY_5_one_hot", "PAY_6_one_hot"))

val dfHotEncoded = encoder.fit(dfScaled).transform(dfScaled)

dfHotEncoded.select("PAY_3_one_hot").show(10)

+---------------+
|  PAY_3_one_hot|
+---------------+
|(11,[10],[1.0])|
| (11,[0],[1.0])|
| (11,[0],[1.0])|
| (11,[0],[1.0])|
|(11,[10],[1.0])|
| (11,[0],[1.0])|
| (11,[0],[1.0])|
|(11,[10],[1.0])|
| (11,[2],[1.0])|
|     (11,[],[])|
+---------------+
only showing top 10 rows



import org.apache.spark.ml.feature.OneHotEncoderEstimator
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_afcecab24842
dfHotEncoded: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 33 more fields]


### Create the final data frame

In [85]:
val va = new VectorAssembler()
    .setInputCols(Array("SCALED_FEATURES", "MARRIAGE_one_hot", "EDUCATION_one_hot", "SEX_one_hot",
                        "PAY_0_one_hot", "PAY_2_one_hot", "PAY_3_one_hot", "PAY_4_one_hot",
                        "PAY_5_one_hot", "PAY_6_one_hot"))
    .setOutputCol("features")

val dataset = va.transform(dfHotEncoded).select("features", "label")

dataset.show(1, false)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                              |label|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(91,[0,1,2,3,4,9,15,19,27,38,57,68],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
only showing top 1 row



va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_aebe383504e0
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]


### Split the data

In [86]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet.show(3)
testSet.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(91,[0,1,2,3,4,5,...|    0|
|(91,[0,1,2,3,4,5,...|    0|
|(91,[0,1,2,3,4,5,...|    0|
+--------------------+-----+
only showing top 3 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(91,[0,1,2,3,4,5,...|    0|
|(91,[0,1,2,3,4,5,...|    0|
|(91,[0,1,2,3,4,5,...|    0|
+--------------------+-----+
only showing top 3 rows



trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]


In [99]:
import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, LogisticRegression}
import org.apache.spark.ml.regression.{LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor}
import org.apache.spark.ml.evaluation.{RegressionEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}

import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, LogisticRegression}
import org.apache.spark.ml.regression.{LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor}
import org.apache.spark.ml.evaluation.{RegressionEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}


### Build and train different models

### Logistic regression model

In [100]:
// Instantiate the model
val lrModel = new LogisticRegression()
    .setMaxIter(50)
    .setFeaturesCol("features")
    .setLabelCol("label")

// Define the hyper-parameter grid
val paramGrid = new ParamGridBuilder()
    .addGrid(lrModel.regParam, Array(0.1, 0.05, 0.01, 0))
    .addGrid(lrModel.elasticNetParam, Array(0.1, 0.05, 0.01, 0))
    .build()

// The BinaryClassificationEvaluator evaluates on the Area Under the ROC-curve (AUC) metric
val evaluator = new BinaryClassificationEvaluator()
val cv = new CrossValidator()
    .setEstimator(lrModel)
    .setEstimatorParamMaps(paramGrid)
    .setEvaluator(evaluator)
    .setNumFolds(5)

// Perform cross-validation for model selection
val cvModel = cv.fit(trainSet)

// Predict the labels of the test set
val predictions = cvModel.transform(testSet)

println("The best logistic regression model has the following attributes:\n")
println(cvModel.bestModel.extractParamMap())

println("\nThe AUC of the best logistic regression model is: ")
println(evaluator.evaluate(predictions))

The best logistic regression model has the following attributes:

{
	logreg_7e71f1361674-aggregationDepth: 2,
	logreg_7e71f1361674-elasticNetParam: 0.1,
	logreg_7e71f1361674-family: auto,
	logreg_7e71f1361674-featuresCol: features,
	logreg_7e71f1361674-fitIntercept: true,
	logreg_7e71f1361674-labelCol: label,
	logreg_7e71f1361674-maxIter: 50,
	logreg_7e71f1361674-predictionCol: prediction,
	logreg_7e71f1361674-probabilityCol: probability,
	logreg_7e71f1361674-rawPredictionCol: rawPrediction,
	logreg_7e71f1361674-regParam: 0.01,
	logreg_7e71f1361674-standardization: true,
	logreg_7e71f1361674-threshold: 0.5,
	logreg_7e71f1361674-tol: 1.0E-6
}

The AUC of the best logistic regression model is: 
0.7692141251631736


lrModel: org.apache.spark.ml.classification.LogisticRegression = logreg_7e71f1361674
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_7e71f1361674-elasticNetParam: 0.1,
	logreg_7e71f1361674-regParam: 0.1
}, {
	logreg_7e71f1361674-elasticNetParam: 0.1,
	logreg_7e71f1361674-regParam: 0.05
}, {
	logreg_7e71f1361674-elasticNetParam: 0.1,
	logreg_7e71f1361674-regParam: 0.01
}, {
	logreg_7e71f1361674-elasticNetParam: 0.1,
	logreg_7e71f1361674-regParam: 0.0
}, {
	logreg_7e71f1361674-elasticNetParam: 0.05,
	logreg_7e71f1361674-regParam: 0.1
}, {
	logreg_7e71f1361674-elasticNetParam: 0.05,
	logreg_7e71f1361674-regParam: 0.05
}, {
	logreg_7e71f1361674-elasticNetParam: 0.05,
	logreg_7e71f1361674-regParam: 0.01
}, {
	logreg_7e71f1361674-elasticNetParam:...

## Build and Train a decision tree classifier

In [101]:
// Instantiate the decision tree classifier
val dtModel = new DecisionTreeClassifier()
    .setLabelCol("label")
    .setFeaturesCol("features")

val paramGrid = new ParamGridBuilder()
    .addGrid(dtModel.impurity, Array("entropy", "gini"))
    .addGrid(dtModel.maxDepth, Array(4, 5, 6, 7, 8, 9, 10))
    .addGrid(dtModel.minInstancesPerNode, Array(1, 2))//, 3))
    .build()

val evaluator = new BinaryClassificationEvaluator()
val cv = new CrossValidator()
    .setEstimator(dtModel)
    .setEstimatorParamMaps(paramGrid)
    .setEvaluator(evaluator)
    .setNumFolds(5)

// Perform cross-validation for model selection
val cvModel = cv.fit(trainSet)

// Predict the labels of the test set
val predictions = cvModel.transform(testSet)

print("The best decision model has the following attributes:\n")
print(cvModel.bestModel.extractParamMap())

print("\nThe AUC of the best decision tree model is: ")
print(1 - evaluator.evaluate(predictions))

The best decision model has the following attributes:
{
	dtc_b26cd928e60f-cacheNodeIds: false,
	dtc_b26cd928e60f-checkpointInterval: 10,
	dtc_b26cd928e60f-featuresCol: features,
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-labelCol: label,
	dtc_b26cd928e60f-maxBins: 32,
	dtc_b26cd928e60f-maxDepth: 10,
	dtc_b26cd928e60f-maxMemoryInMB: 256,
	dtc_b26cd928e60f-minInfoGain: 0.0,
	dtc_b26cd928e60f-minInstancesPerNode: 1,
	dtc_b26cd928e60f-predictionCol: prediction,
	dtc_b26cd928e60f-probabilityCol: probability,
	dtc_b26cd928e60f-rawPredictionCol: rawPrediction,
	dtc_b26cd928e60f-seed: 159147643
}
The AUC of the best decision tree model is: 0.4947864891586592

dtModel: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_b26cd928e60f
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-maxDepth: 4,
	dtc_b26cd928e60f-minInstancesPerNode: 1
}, {
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-maxDepth: 5,
	dtc_b26cd928e60f-minInstancesPerNode: 1
}, {
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-maxDepth: 6,
	dtc_b26cd928e60f-minInstancesPerNode: 1
}, {
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-maxDepth: 7,
	dtc_b26cd928e60f-minInstancesPerNode: 1
}, {
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f-maxDepth: 8,
	dtc_b26cd928e60f-minInstancesPerNode: 1
}, {
	dtc_b26cd928e60f-impurity: entropy,
	dtc_b26cd928e60f...

### Build and Train a random forest classifier

In [112]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.classification.RandomForestClassifier

val rfModel = new RandomForestClassifier()
    .setNumTrees(50)
    .setLabelCol("label")
    .setFeaturesCol("features")

val paramGrid = new org.apache.spark.ml.tuning.ParamGridBuilder()
    .addGrid(rfModel.featureSubsetStrategy, Array("auto", "all", "sqrt"))
    .addGrid(rfModel.minInstancesPerNode, Array(1, 2, 3))
    .build()

val evaluator = new BinaryClassificationEvaluator()
val cv = new CrossValidator()
    .setEstimator(rfModel)
    .setEstimatorParamMaps(paramGrid)
    .setEvaluator(evaluator)
    .setNumFolds(5)

// Perform cross-validation for model selection
val cvModel = cv.fit(trainSet)

// Predict the labels of the test set
val predictions = cvModel.transform(testSet)

print("The best random forest model has the following attributes:\n")
print(cvModel.bestModel.extractParamMap())

print("\nThe AUC of the best random forest model is: ")
print(evaluator.evaluate(predictions))

The best random forest model has the following attributes:
{
	rfc_53dd480228b4-cacheNodeIds: false,
	rfc_53dd480228b4-checkpointInterval: 10,
	rfc_53dd480228b4-featureSubsetStrategy: auto,
	rfc_53dd480228b4-featuresCol: features,
	rfc_53dd480228b4-impurity: gini,
	rfc_53dd480228b4-labelCol: label,
	rfc_53dd480228b4-maxBins: 32,
	rfc_53dd480228b4-maxDepth: 5,
	rfc_53dd480228b4-maxMemoryInMB: 256,
	rfc_53dd480228b4-minInfoGain: 0.0,
	rfc_53dd480228b4-minInstancesPerNode: 2,
	rfc_53dd480228b4-numTrees: 50,
	rfc_53dd480228b4-predictionCol: prediction,
	rfc_53dd480228b4-probabilityCol: probability,
	rfc_53dd480228b4-rawPredictionCol: rawPrediction,
	rfc_53dd480228b4-seed: 207336481,
	rfc_53dd480228b4-subsamplingRate: 1.0
}
The AUC of the best random forest model is: 0.7614917644654685

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.classification.RandomForestClassifier
rfModel: org.apache.spark.ml.classification.RandomForestClassifier = rfc_53dd480228b4
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_53dd480228b4-featureSubsetStrategy: auto,
	rfc_53dd480228b4-minInstancesPerNode: 1
}, {
	rfc_53dd480228b4-featureSubsetStrategy: auto,
	rfc_53dd480228b4-minInstancesPerNode: 2
}, {
	rfc_53dd480228b4-featureSubsetStrategy: auto,
	rfc_53dd480228b4-minInstancesPerNode: 3
}, {
	rfc_53dd480228b4-featureSubsetStrategy: all,
	rfc_53dd480228b4-minInstancesPerNode: 1
}, {
	rfc_53dd480228b4-featureSubsetStrategy: all,
	rfc_53dd480228b4-minInstancesPerNode: 2
}, {
	rfc_53dd480228b4-featureSubsetStrategy: all,
	r...